## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Bluff,-46.60,168.33,45.00,72,36,11.01,NZ,scattered clouds
1,1,Thompson,55.74,-97.86,51.80,93,90,6.93,CA,overcast clouds
2,2,Port Alfred,-33.59,26.89,59.61,68,0,10.96,ZA,clear sky
3,3,Namibe,-15.20,12.15,61.86,80,25,11.99,AO,scattered clouds
4,4,Tabou,4.42,-7.35,78.06,81,94,12.39,CI,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
4,4,Tabou,4.42,-7.35,78.06,81,94,12.39,CI,overcast clouds
5,5,Trincomalee,8.57,81.23,82.02,71,82,8.30,LK,broken clouds
6,6,Cabo San Lucas,22.89,-109.91,78.01,88,90,2.24,MX,overcast clouds
8,8,Butaritari,3.07,172.79,82.63,72,99,13.11,KI,overcast clouds
14,14,Atuona,-9.80,-139.03,77.86,79,43,20.67,PF,light rain
15,15,Jacqueville,5.21,-4.41,80.60,78,75,11.41,CI,broken clouds
16,16,Sao Filipe,14.90,-24.50,79.07,82,89,4.14,CV,light rain
21,21,Luanda,-8.84,13.23,75.20,78,40,10.29,AO,scattered clouds
27,27,Pathein,16.78,94.73,79.12,90,96,4.45,MM,overcast clouds
31,31,Isangel,-19.55,169.27,74.44,84,19,13.76,VU,light rain


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                250
City                   250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Country                249
Current Description    250
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
4,4,Tabou,4.42,-7.35,78.06,81,94,12.39,CI,overcast clouds
5,5,Trincomalee,8.57,81.23,82.02,71,82,8.30,LK,broken clouds
6,6,Cabo San Lucas,22.89,-109.91,78.01,88,90,2.24,MX,overcast clouds
8,8,Butaritari,3.07,172.79,82.63,72,99,13.11,KI,overcast clouds
14,14,Atuona,-9.80,-139.03,77.86,79,43,20.67,PF,light rain
...,...,...,...,...,...,...,...,...,...,...
674,674,Champerico,14.30,-91.92,82.40,74,40,6.93,GT,scattered clouds
676,676,Manado,1.49,124.85,75.20,94,20,3.31,ID,few clouds
679,679,Kahului,20.89,-156.47,77.00,69,20,8.05,US,few clouds
682,682,Tongren,27.72,109.19,74.97,70,0,0.85,CN,light rain


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tabou,CI,78.06,overcast clouds,4.42,-7.35,
5,Trincomalee,LK,82.02,broken clouds,8.57,81.23,
6,Cabo San Lucas,MX,78.01,overcast clouds,22.89,-109.91,
8,Butaritari,KI,82.63,overcast clouds,3.07,172.79,
14,Atuona,PF,77.86,light rain,-9.80,-139.03,
15,Jacqueville,CI,80.60,broken clouds,5.21,-4.41,
16,Sao Filipe,CV,79.07,light rain,14.90,-24.50,
21,Luanda,AO,75.20,scattered clouds,-8.84,13.23,
27,Pathein,MM,79.12,overcast clouds,16.78,94.73,
31,Isangel,VU,74.44,light rain,-19.55,169.27,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tabou,CI,78.06,overcast clouds,4.42,-7.35,hotêl doufoulougou
5,Trincomalee,LK,82.02,broken clouds,8.57,81.23,Welcombe Hotel
6,Cabo San Lucas,MX,78.01,overcast clouds,22.89,-109.91,Hotel Tesoro Los Cabos
8,Butaritari,KI,82.63,overcast clouds,3.07,172.79,Isles Sunset Lodge
14,Atuona,PF,77.86,light rain,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
674,Champerico,GT,82.40,scattered clouds,14.30,-91.92,"Hotel y restaurante ""El Diamante"""
676,Manado,ID,75.20,few clouds,1.49,124.85,Aryaduta Hotel
679,Kahului,US,77.00,few clouds,20.89,-156.47,Maui Seaside Hotel
682,Tongren,CN,74.97,light rain,27.72,109.19,Fanjing Mountain International Conference Center


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))